In [1]:
print ('Name: Leonard Umoru \nStudent Number: 041152507')

Name: Leonard Umoru 
Student Number: 041152507


In [3]:
import json #For handling file input/output


class Book: # Class to represent books
    def __init__(self, title, author, isbn):
        self.title = title
        self.author = author   # initializing attributes
        self.isbn = isbn
        self.available = True

    def __str__(self): #String method for showing how the books should be displayed
        if self.available: # if in library
            return f'({self.title} by {self.author}, ISBN: {self.isbn}, Availability: "Available")'
        else: # if not in library
            return f'({self.title} by {self.author}, ISBN: {self.isbn}, Availability: "Unavailable")'


class UserClass: #Class to represent regular users
    def __init__(self, name, user_id):
        self.name = name
        self.user_id = user_id  #Initializing attributes
        self.borrowed_books = []  #List for storing user's borrowed books

    def borrow_book(self, book): #Method to borrow a book
        while len(self.borrowed_books) >= 3: #While loop to limit regular users from borrowing more than 3 books
            print(f'{self.name} has reached their borrow limit')
            break #STOPS THE INFINITE LOOP OF LIMIT MESSAGE
        else:
            if book.available: #Checks if book is avaialable for borrowing
                book.available = False #Marks the book as unavailable
                self.borrowed_books.append(book)  #adds the book the user's borrowed list
                print(f'{self.name} has successfully borrowed the book, {book.title}')
            else:
                print(f'Sorry, the book, {book.title} is not available.') #if the book is unavailable

    def return_book(self, book): #Method to return a book
        if book in self.borrowed_books: #Checks if book is in user's borrowed list
            book.available = True # Marks the book back to available
            self.borrowed_books.remove(book) #Removes the book for the user's borrowed list
            print(f'{self.name} has returned the book, {book.title}')
        else:
            print(f'{self.name} does not have the book, {book.title}.') #If the user didnt borrow the book
            
    def __str__(self): #String method to represent how the users and their books are displayed
        borrowed_titles = ', '.join(str(book) for book in self.borrowed_books) #converts the books in the list into strings and seperates them by ','.
        return f'User {self.name}, borrowed these books: [{borrowed_titles}]'


class Library: #Class to represent the library itself
    def __init__(self):
        self.books = []  #List to store books in the library
        self.users = []  #List to store users in the library 

    def add_book(self, book):  #method for adding books to the library
        self.books.append(book) #adds book to the library's list of books
        print(f'{book.title} added to the library.')

    def add_user(self, user):  #method for adding users to the library
        self.users.append(user) #adds users to the library's list of users
        print(f'{user.name} added to the library.')

    def borrow_book(self, user_id, isbn): #Method for users to borrow books using the book's ISBN
        user = next((u for u in self.users if u.user_id == user_id), None) #Generator expression to find the user by thier ID
        book = next((b for b in self.books if b.isbn == isbn), None) #Generator expression to find the book by its ISBN
        if user and book: 
            user.borrow_book(book) #Allows users to borrow books only if both user and book exists
        else:
            if not user:
                print(f'Cannot find User with ID {user_id}.') #If user doesnt exist
            if not book:
                print(f'Cannot find Book with ISBN: {isbn}.') #If book doesnt exist

    def return_book(self, user_id, isbn): #Method for users to return books using the book's ISBN
        user = next((u for u in self.users if u.user_id == user_id), None) #Generator expression to find the user by their ID
        book = next((b for b in self.books if b.isbn == isbn), None) #Generator expression to find the book by its ISBN
        if user and book:
            user.return_book(book) #Allows users to return books only if both user and book exists
        else:
            if not user:
                print(f'Cannot find User with ID {user_id}.')  #If user doesnt exist
            if not book:
                print(f'Cannot find Book with ISBN: {isbn}.')  #If book doesnt exist
    
    def display_books(self): #method to display all books in the library
        for book in self.books: #loops through the book list and prints each one
            print(book)

    def display_users(self): #method to display all users in the library
        for user in self.users: #loops through the user list and prints each one
            print(user)

    def file_input(self, filename = 'library.json'): #Method to save library's information (books and users) to a file
        data = {  #Creating a dictionary that holds two main keys (books and users)
            'books': [
                {
                    'title':book.title,
                    'author':book.author,
                    'isbn':book.isbn,
                    'available':book.available
                } for book in self.books
            ],
            
            'users': [
                {
                    'name':user.name,
                    'user_id':user.user_id,
                    'borrowed_books':[book.isbn for book in user.borrowed_books] #saves ISBNs of borrowed books
                } for user in self.users #converts each user to a dictionary
            ]
        }
        
        with open(filename, 'w') as file:
            json.dump(data, file) # Write the library data to a JSON file
        print('Library information has been saved to file.')
        

    def file_output(self, filename = 'library.json'): #Method to load library's information(books and users from a file
        try:
            with open(filename, 'r') as file:
                data = json.load(file) # Read the library data from a JSON file
                
                self.books = [Book(book['title'], book['author'], book['isbn']) for book in data['books']] #Recreate book objects
                for bk, book_info in zip(self.books, data['books']): #This loop pairs each Book object in self.books with its corresponding dictionary from data['books']
                    bk.available = book_info['available'] #Sets the availability of each book

                self.users = [UserClass(user['name'], user['user_id']) for user in data['users']] #Recreate user objects
                for usr, user_info in zip(self.users, data['users']): #This loop pairs each UserClass object in self.users with its corresponding dictionary from data['users']
                    usr.borrowed_books = [next((b for b in self.books if b.isbn == isbn), None) for isbn in user_info['borrowed_books']] #Restore the borrowed books
            print('Library information has been loaded from file.')
            
        except FileNotFoundError:
            print('No Library information was found.') #Error message if the file doesnt exist
            


class PremiumUser(UserClass): #Subclass for premium users who can borrow up to 5 books instead of 3 like the regular users
    def borrow_book(self, book):
        while len(self.borrowed_books) >= 5: #While loop to limit premium users from borrowing more than 5 books
            print(f'{self.name} has reached their borrow limit')
            break #stops infinite loop
        else:
            if book.available: #Checks if book is avaialable for borrowing
                book.available = False #Marks the book as unavailable
                self.borrowed_books.append(book) #adds the book the user's borrowed list
                print(f'{self.name} has successfully borrowed the book, {book.title}')
            else:
                print(f'Sorry, the book, {book.title} is not available.')


#Feel free to rename the json file in my 'file_input' and 'file_output' to test on a fresh file, and run the below codes in the sequence as is.

In [4]:
# Create a library instance
library = Library()

In [7]:
# Load existing data from file and should give the 'no library information found' error message 
#because we havnt saved to file yet
library.file_output()

No Library information was found.


In [9]:
# Create 10 books
book1 = Book('Economics', 'Michael Jordan', 1234267332)
book2 = Book('Sociology', 'Jalen Green', 5342312137)
book3 = Book('Engineering', 'Caitlyn Clark', 1134565653)
book4 = Book('Biology', 'Jimmy Butler', 1134452223)
book5 = Book('Chemistry', 'Lamelo Ball', 1928748223)
book6 = Book('Physics', 'Lebron James', 1817981212)
book7 = Book('Geography', 'Stephen Curry', 8547228915)
book8 = Book('History', 'Angel Reese', 7887288321)
book9 = Book('Mathematics', 'Jason Tatum', 9738203091)
book10 = Book('Computer Science', 'James Harden', 6782478201)

In [11]:
# Added the 10 books to library
library.add_book(book1)
library.add_book(book2)
library.add_book(book3)
library.add_book(book4)
library.add_book(book5)
library.add_book(book6)
library.add_book(book7)
library.add_book(book8)
library.add_book(book9)
library.add_book(book10)

Economics added to the library.
Sociology added to the library.
Engineering added to the library.
Biology added to the library.
Chemistry added to the library.
Physics added to the library.
Geography added to the library.
History added to the library.
Mathematics added to the library.
Computer Science added to the library.


In [13]:
# Create 5 users
user1 = UserClass('Roland', 1)
user2 = UserClass('Annette', 2)
user3 = UserClass('Gerald', 3)
user4 = UserClass('Sally', 4)
user5 = PremiumUser('Leonard', 5)

In [15]:
# Add 5 users to library
library.add_user(user1)
library.add_user(user2)
library.add_user(user3)
library.add_user(user4)
library.add_user(user5)

Roland added to the library.
Annette added to the library.
Gerald added to the library.
Sally added to the library.
Leonard added to the library.


In [17]:
# Display all avalilable books in library
library.display_books()

(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Available")
(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Available")
(Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Available")
(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Available")
(Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Available")
(Physics by Lebron James, ISBN: 1817981212, Availability: "Available")
(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Available")
(History by Angel Reese, ISBN: 7887288321, Availability: "Available")
(Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Available")
(Computer Science by James Harden, ISBN: 6782478201, Availability: "Available")


In [19]:
# Display all users in the libary
library.display_users()

User Roland, borrowed these books: []
User Annette, borrowed these books: []
User Gerald, borrowed these books: []
User Sally, borrowed these books: []
User Leonard, borrowed these books: []


In [21]:
#Borrow books from the library showing regular and premium benefits. Here Leonard is a Premium User, while Roland
#is a regular user
library.borrow_book(5, 1234267332)
library.borrow_book(5, 5342312137)
library.borrow_book(5, 1134565653)
library.borrow_book(5, 1134452223)
library.borrow_book(5, 1928748223)
library.borrow_book(5, 1817981212)
library.borrow_book(1, 8547228915)
library.borrow_book(1, 7887288321)
library.borrow_book(1, 9738203091)
library.borrow_book(1, 6782478201)

Leonard has successfully borrowed the book, Economics
Leonard has successfully borrowed the book, Sociology
Leonard has successfully borrowed the book, Engineering
Leonard has successfully borrowed the book, Biology
Leonard has successfully borrowed the book, Chemistry
Leonard has reached their borrow limit
Roland has successfully borrowed the book, Geography
Roland has successfully borrowed the book, History
Roland has successfully borrowed the book, Mathematics
Roland has reached their borrow limit


In [23]:
#Run this, note all borrowed books have now become unavailable
library.display_books()

(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Unavailable")
(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Unavailable")
(Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Unavailable")
(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Unavailable")
(Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")
(Physics by Lebron James, ISBN: 1817981212, Availability: "Available")
(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")
(History by Angel Reese, ISBN: 7887288321, Availability: "Unavailable")
(Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Unavailable")
(Computer Science by James Harden, ISBN: 6782478201, Availability: "Available")


In [25]:
#Run this, note the books that these users borrowed are now in their list of borrowed books
library.display_users()

User Roland, borrowed these books: [(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable"), (History by Angel Reese, ISBN: 7887288321, Availability: "Unavailable"), (Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Unavailable")]
User Annette, borrowed these books: []
User Gerald, borrowed these books: []
User Sally, borrowed these books: []
User Leonard, borrowed these books: [(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Unavailable"), (Sociology by Jalen Green, ISBN: 5342312137, Availability: "Unavailable"), (Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Unavailable"), (Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Unavailable"), (Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")]


In [27]:
#Returning books back to the library
library.return_book(5, 1234267332)
library.return_book(5, 5342312137)
library.return_book(5, 1134565653)
library.return_book(1, 7887288321)
library.return_book(1, 9738203091)

Leonard has returned the book, Economics
Leonard has returned the book, Sociology
Leonard has returned the book, Engineering
Roland has returned the book, History
Roland has returned the book, Mathematics


In [29]:
#Updated book lists
library.display_books()

(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Available")
(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Available")
(Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Available")
(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Unavailable")
(Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")
(Physics by Lebron James, ISBN: 1817981212, Availability: "Available")
(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")
(History by Angel Reese, ISBN: 7887288321, Availability: "Available")
(Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Available")
(Computer Science by James Harden, ISBN: 6782478201, Availability: "Available")


In [31]:
#Updated user list
library.display_users()

User Roland, borrowed these books: [(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")]
User Annette, borrowed these books: []
User Gerald, borrowed these books: []
User Sally, borrowed these books: []
User Leonard, borrowed these books: [(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Unavailable"), (Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")]


In [33]:
#Save new data to file
library.file_input()

Library information has been saved to file.


In [35]:
#Now it should load last saved data from the file
library.file_output()

Library information has been loaded from file.


In [37]:
#Lets try performing some actions in the library without saving and loading again
library.borrow_book(2, 5342312137)
library.return_book(5, 1134452223)

Annette has successfully borrowed the book, Sociology
Leonard has returned the book, Biology


In [39]:
#note changes made, Sociology changed to Unavailable and Biology changed to Available and Roland, Annette and Leonard
#each have one book in their borrowed list
library.display_books()
library.display_users()

(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Available")
(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Unavailable")
(Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Available")
(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Available")
(Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")
(Physics by Lebron James, ISBN: 1817981212, Availability: "Available")
(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")
(History by Angel Reese, ISBN: 7887288321, Availability: "Available")
(Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Available")
(Computer Science by James Harden, ISBN: 6782478201, Availability: "Available")
User Roland, borrowed these books: [(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")]
User Annette, borrowed these books: [(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Unavailable")]
User Gerald, borrowed these books:

In [41]:
#Run this
library.file_output()

Library information has been loaded from file.


In [43]:
#Then run this again, note how it reverted back to the data it last saved, note how the last actions didnt go through
#because the file wasnt saved
library.display_books()
library.display_users()

(Economics by Michael Jordan, ISBN: 1234267332, Availability: "Available")
(Sociology by Jalen Green, ISBN: 5342312137, Availability: "Available")
(Engineering by Caitlyn Clark, ISBN: 1134565653, Availability: "Available")
(Biology by Jimmy Butler, ISBN: 1134452223, Availability: "Unavailable")
(Chemistry by Lamelo Ball, ISBN: 1928748223, Availability: "Unavailable")
(Physics by Lebron James, ISBN: 1817981212, Availability: "Available")
(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")
(History by Angel Reese, ISBN: 7887288321, Availability: "Available")
(Mathematics by Jason Tatum, ISBN: 9738203091, Availability: "Available")
(Computer Science by James Harden, ISBN: 6782478201, Availability: "Available")
User Roland, borrowed these books: [(Geography by Stephen Curry, ISBN: 8547228915, Availability: "Unavailable")]
User Annette, borrowed these books: []
User Gerald, borrowed these books: []
User Sally, borrowed these books: []
User Leonard, borrowed these boo